In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key="13b86763ab8ddf529c91c7dce385c6cb04b5253e")

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import matplotlib.image as mpimg
import wandb
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import random_split

In [ ]:
from torchvision import transforms, datasets
train_set = "/kaggle/input/inature12k/inaturalist_12K/train"
test_set = "/kaggle/input/inature12k/inaturalist_12K/val"
IMG_SIZE = (224,224)
# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize the images
    transforms.ToTensor(),        # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])  # Normalize the images


# Use ImageFolder to create a dataset
trainset = datasets.ImageFolder(root=train_set, transform=transform)
val_size = int(0.2 * len(trainset))
train_size = len(trainset) - val_size
trainset, valset = random_split(trainset, [train_size, val_size])
testset = datasets.ImageFolder(root=test_set, transform=transform)

In [ ]:
class Convolutional_Neural_Network(nn.Module):
    def __init__(self, trainset, valset, batch_size , num_filters,filter_org, kernel_size,
                 act_fn, num_neurons, batch_norm, dropout_rate):
            super().__init__()
            
            # DataLoader for the dataset
            self.dataloader_train = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True)
            self.dataloader_val = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle=True)

            self.batch_norm = batch_norm
            filters = []
            if filter_org=='same':
                filters = [num_filters]*5
            elif filter_org=='double':
                filters = [num_filters, num_filters*2, num_filters*4, num_filters*8, num_filters*16]
            elif filter_org=='half':
                filters = [num_filters, num_filters//2, num_filters//4, num_filters//8, num_filters//16]
        #Convolution layer 1
            self.conv_1 = nn.Conv2d(3, filters[0],
                               kernel_size=(kernel_size[0],kernel_size[0]),
                               stride=1, padding='same')
            x = 224          
            self.bn_1 = nn.BatchNorm2d(filters[0])  
            self.act_1 = self.activation_function(act_fn)
            self.dropout_1 = nn.Dropout(p=dropout_rate)
            self.pool_1 = nn.MaxPool2d(2,2)
            x = x//2
            self.conv_2 = nn.Conv2d(filters[0], filters[1],
                               kernel_size=(kernel_size[1],kernel_size[1]),
                               stride=1, padding='same')
            self.bn_2 = nn.BatchNorm2d(filters[1])
            self.act_2 = self.activation_function(act_fn)
            self.dropout_2 = nn.Dropout(p=dropout_rate)
            self.pool_2 = nn.MaxPool2d(2,2)
   

        #Convolution layer 3
            self.conv_3 = nn.Conv2d(filters[1], filters[2],
                               kernel_size=(kernel_size[2],kernel_size[2]),
                               stride=1, padding='same')

            self.bn_3 = nn.BatchNorm2d(filters[2])
            self.act_3 = self.activation_function(act_fn)
            self.dropout_3 = nn.Dropout(p=dropout_rate)
            self.pool_3 = nn.MaxPool2d(2,2)


        #Convolution layer 4
            self.conv_4 = nn.Conv2d(filters[2], filters[3],
                               kernel_size=(kernel_size[3],kernel_size[3]),
                               stride=1, padding='same')
            self.bn_4 = nn.BatchNorm2d(filters[3])
            self.act_4 = self.activation_function(act_fn)
            self.dropout_4 = nn.Dropout(p=dropout_rate)
            self.pool_4 = nn.MaxPool2d(2,2)
        

        #Convolution layer 5
            self.conv_5 = nn.Conv2d(filters[3], filters[4],
                               kernel_size=(kernel_size[4],kernel_size[4]),
                               stride=1, padding='same')
            self.bn_5 = nn.BatchNorm2d(filters[4])
            self.act_5 = self.activation_function(act_fn)
            self.dropout_5 = nn.Dropout(p=dropout_rate)
            self.pool_5 = nn.MaxPool2d(2,2)
            x = x//(2**5)
            x = x*x*filters[4]

            self.flatten = nn.Flatten()

        # dense layer
            self.fc_6 = nn.Linear(x, num_neurons)
            self.act_6 = self.activation_function(act_fn)
            self.dropout_6 = nn.Dropout(p=dropout_rate)
        
        # output layer
            self.fc_7 = nn.Linear(num_neurons, 10)

    def activation_function(self,act_fn):
            if act_fn=='relu':
              return nn.ReLU()
            elif act_fn=='tanh':
              return nn.Tanh()
            elif act_fn=='gelu':
              return nn.GELU()
            elif act_fn=='silu':
              return nn.SiLU()
            elif act_fn=='mish':
              return nn.Mish()
        
    def forward(self, x):
        
        x = self.conv_1(x)
        if self.batch_norm:
            x = self.bn_1(x)
        x = self.act_1(x)
        x = self.dropout_1(x)
        x = self.pool_1(x)

 
        x = self.conv_2(x)
        if self.batch_norm:
            x = self.bn_2(x)
        x = self.act_2(x)
        x = self.dropout_2(x)

        x = self.pool_2(x)

  
        x = self.conv_3(x)
        if self.batch_norm:
            x = self.bn_3(x)
        x = self.act_3(x)
        x = self.dropout_3(x)
        x = self.pool_3(x)


        x = self.conv_4(x)
        if self.batch_norm:
            x = self.bn_4(x)
        x = self.act_4(x)
        x = self.dropout_4(x)

        x = self.pool_4(x)

        x = self.conv_5(x)
        if self.batch_norm:
            x = self.bn_5(x)
        x = self.act_5(x)
        x = self.dropout_5(x)
        x = self.pool_5(x)


        x = self.flatten(x)
        x = self.act_6(self.fc_6(x))
        x = self.dropout_6(x)
        x = self.fc_7(x)
        return x

    def train_model(self, model, loss_fn, optimizer, n_epochs = 10):
        for epoch in range(n_epochs):
            train_accuracy = 0
            count = 0
            for inputs, labels in self.dataloader_train:
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Forward pass
                y_pred = model(inputs)
                train_loss = loss_fn(y_pred, labels)
                train_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
                count += len(labels)
                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()
            train_accuracy /= count
            print("Epoch %d: model train accuracy %.2f%%" % (epoch, train_accuracy*100))
            wandb.log({ 'epoch': epoch, 'train_accuracy': train_accuracy * 100})
            wandb.log({ 'epoch': epoch, 'train_loss': train_loss * 100})
            
            val_accuracy = 0
            count = 0
            for inputs, labels in self.dataloader_val:
                inputs = inputs.to(device)
                labels = labels.to(device)
                y_pred = model(inputs)
                val_loss = loss_fn(y_pred, labels)
                val_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
                count += len(labels)
            val_accuracy /= count
            print("Epoch %d: model validation accuracy %.2f%%" % (epoch, val_accuracy*100))
            wandb.log({ 'epoch': epoch, 'validation_accuracy': val_accuracy * 100})
            wandb.log({ 'epoch': epoch, 'validation_loss': val_loss * 100})
        return model

In [22]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [12]:
model = Convolutional_Neural_Network(trainset, valset, batch_size= 64, num_filters = 32 ,
                filter_org ="half" , kernel_size = [3, 5, 5, 7, 7] , act_fn ='relu',
                num_neurons = 128 ,batch_norm ="True", dropout_rate=0.2)


In [22]:
loss_fn = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay= 0.0005)
model.train_model(model, loss_fn, optimizer, n_epochs = 1)

Epoch 0: model train accuracy 16.73%
Epoch 0: model validation accuracy 20.11%


Convolutional_Neural_Network(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (dropout1): Dropout(p=0.2, inplace=False)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 16, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU()
  (dropout2): Dropout(p=0.2, inplace=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 8, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (bn3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act3): ReLU()
  (dropout3): Dropout(p=0.2, inplace=False)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(8, 4, kernel_size=(7, 

In [7]:
# sweep config file
sweep_config = {
    'method': 'bayes',
    'name' : 'bayes sweep',
    'metric': {
      'goal': 'maximize',
      'name': 'validation_accuracy'
    },
    'parameters': {
        'kernel_size':{
            'values': [[3]*5, [3, 5, 5, 7, 7], [5]*5, [7]*7, [7, 5, 5, 3, 3] ]
        },
        'num_filter':{
            'values': [32, 64, 128]
        },
        'filter_organisation':{
          'values': ['same', 'half','double']
        },
        'activation_fn': {
            'values': ['relu', 'gelu', 'silu', 'mish', 'tanh']
        },
        'data_aug':{
            'values': [False]
        },
        'batch_norm':{
            'values': [True, False]
        },
        'dropout': {
            'values': [0, 0.2, 0.5]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'L2_regularisation':{
            'values': [0, 0.0005, 0.05]
        },
        'batch_size': {
            'values': [32, 64]
        }
,
        'neurons_dense':{
            'values': [64, 128, 256]
        },
        'epochs':{
            'values': [10,15,20]
        }
    }
}

In [10]:
sweep_id = wandb.sweep(sweep = sweep_config, project='DA6401-Assignment_2')

Create sweep with ID: s11qip6f
Sweep URL: https://wandb.ai/iitm-ma23m015/DA6401-Assignment_2/sweeps/s11qip6f


In [24]:
sweep_id = 'v0xii055'

In [11]:
def main():
  with wandb.init() as run:
    run_name = "-f_num_"+str(wandb.config.num_filter)+"-f_num_"+wandb.config.filter_organisation+"-ac_fn_"+wandb.config.activation_fn+\
                "-b_norm_"+str(wandb.config.batch_norm) + "-bs_"+str(wandb.config.batch_size) +"-neu_num"+str(wandb.config.neurons_dense)

    wandb.run.name = run_name

    model = Convolutional_Neural_Network(trainset, valset, batch_size=wandb.config.batch_size, num_filters = wandb.config.num_filter,
                filter_org =wandb.config.filter_organisation , kernel_size = wandb.config.kernel_size, act_fn = wandb.config.activation_fn,
                num_neurons = wandb.config.neurons_dense,batch_norm = wandb.config.batch_norm, dropout_rate=wandb.config.dropout)
    model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate, weight_decay=wandb.config.L2_regularisation)
    model.train_model(model, loss_fn, optimizer, n_epochs = wandb.config.epochs) # wandb.config.epochs)
wandb.agent(sweep_id, function = main, count = 1) # calls main function for count number of times
wandb.finish()

wandb: Agent Starting Run: fdbza8ip with config:
wandb: 	L2_regularisation: 0.0005
wandb: 	activation_fn: relu
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_aug: False
wandb: 	dropout: 0
wandb: 	epochs: 15
wandb: 	filter_organisation: half
wandb: 	kernel_size: [3, 3, 3, 3, 3]
wandb: 	learning_rate: 0.0001
wandb: 	neurons_dense: 128
wandb: 	num_filter: 64
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 0: model train accuracy 17.90%
Epoch 0: model validation accuracy 21.01%
Epoch 1: model train accuracy 23.90%
Epoch 1: model validation accuracy 25.46%
Epoch 2: model train accuracy 26.65%
Epoch 2: model validation accuracy 26.86%
Epoch 3: model train accuracy 27.76%
Epoch 3: model validation accuracy 27.96%
Epoch 4: model train accuracy 29.86%
Epoch 4: model validation accuracy 28.26%
Epoch 5: model train accuracy 30.75%
Epoch 5: model validation accuracy 30.57%
Epoch 6: model train accuracy 32.34%
Epoch 6: model validation accuracy 31.17%
Epoch 7: model train accuracy 33.86%
Epoch 7: model validation accuracy 31.87%
Epoch 8: model train accuracy 34.18%
Epoch 8: model validation accuracy 32.47%
Epoch 9: model train accuracy 35.79%
Epoch 9: model validation accuracy 32.67%
Epoch 10: model train accuracy 36.09%
Epoch 10: model validation accuracy 31.57%
Epoch 11: model train accuracy 37.89%
Epoch 11: model validation accuracy 31.97%
Epoch 12: model train accuracy 38.41%
Epoch 12: 

epoch,▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇█
train_accuracy,▁▃▄▄▅▅▆▆▆▇▇▇▇██
train_loss,█▇▇▇█▅▅▅▄▄▇▃▁▂▂
validation_accuracy,▁▃▄▅▅▆▆▇▇▇▇▇███
validation_loss,▇█▇▃▆▇▅▃▄▁▃▂▅▄▃
epoch,14
train_accuracy,40.0625
train_loss,159.03679
validation_accuracy,34.06704
validation_loss,163.64005


In [14]:
# Recreate the model with best hyperparameters
best_model = Convolutional_Neural_Network(trainset, valset, batch_size=64, num_filters=32,
                filter_org='double', kernel_size=[7]*5, act_fn='gelu',
                num_neurons=128, batch_norm=True, dropout_rate=0)
best_model.to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=1e-3, weight_decay=0)


import wandb
wandb.init(project="DA6401-Assignment_2", name="best_model_run")


best_model.train_model(best_model, loss_fn, optimizer, n_epochs=15)

Epoch 0: model train accuracy 13.35%
Epoch 0: model validation accuracy 14.91%
Epoch 1: model train accuracy 18.46%
Epoch 1: model validation accuracy 17.76%
Epoch 2: model train accuracy 21.40%
Epoch 2: model validation accuracy 21.66%
Epoch 3: model train accuracy 23.40%
Epoch 3: model validation accuracy 24.06%
Epoch 4: model train accuracy 24.51%
Epoch 4: model validation accuracy 25.61%
Epoch 5: model train accuracy 27.19%
Epoch 5: model validation accuracy 28.56%
Epoch 6: model train accuracy 28.10%
Epoch 6: model validation accuracy 29.21%
Epoch 7: model train accuracy 29.33%
Epoch 7: model validation accuracy 30.27%
Epoch 8: model train accuracy 30.76%
Epoch 8: model validation accuracy 28.26%
Epoch 9: model train accuracy 32.01%
Epoch 9: model validation accuracy 30.57%
Epoch 10: model train accuracy 32.92%
Epoch 10: model validation accuracy 32.07%
Epoch 11: model train accuracy 34.06%
Epoch 11: model validation accuracy 32.37%
Epoch 12: model train accuracy 34.95%
Epoch 12: 

Convolutional_Neural_Network(
  (conv1): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), padding=same)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): GELU(approximate='none')
  (dropout1): Dropout(p=0, inplace=False)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(7, 7), stride=(1, 1), padding=same)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): GELU(approximate='none')
  (dropout2): Dropout(p=0, inplace=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1), padding=same)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act3): GELU(approximate='none')
  (dropout3): Dropout(p=0, inplace=False)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_

In [16]:
# Evaluate test accuracy first
def evaluate_test_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return correct, total

# Create test dataloader
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Call function
correct, total = evaluate_test_accuracy(best_model, testloader)


In [21]:
wandb.log({"Test Accuracy": 100 * correct / total})
print("Test Accuracy:", 100 * correct / total)

Test Accuracy: 33.8


In [26]:
import matplotlib.pyplot as plt
import numpy as np
import wandb
import torch

# Set model to eval mode
best_model.eval()

# Try to extract class names (if available from the dataset)
try:
    class_names = trainset.dataset.classes
except AttributeError:
    class_names = None  # Fallback if class names are not available

# Get 10 random samples from the test set
images, labels = next(iter(torch.utils.data.DataLoader(testset, batch_size=10, shuffle=True)))
images, labels = images.to(device), labels.to(device)

# Predict with the model
with torch.no_grad():
    outputs = best_model(images)
    _, preds = torch.max(outputs, 1)

# Move images, labels, and predictions to CPU for plotting
images = images.cpu()
labels = labels.cpu()
preds = preds.cpu()

# Plot a 10x3 grid: Image | Prediction | Ground Truth
fig, axs = plt.subplots(10, 3, figsize=(6, 20))
fig.suptitle("10×3 Prediction Grid — Best Model", fontsize=16)

for i in range(10):
    # Show image
    axs[i, 0].imshow(np.transpose(images[i], (1, 2, 0)))
    axs[i, 0].axis('off')

    # Prediction text
    pred_label = class_names[preds[i]] if class_names else preds[i].item()
    axs[i, 1].text(0.5, 0.5, str(pred_label), fontsize=12, ha='center', va='center')
    axs[i, 1].axis('off')

    # Ground truth text
    true_label = class_names[labels[i]] if class_names else labels[i].item()
    axs[i, 2].text(0.5, 0.5, str(true_label), fontsize=12, ha='center', va='center')
    axs[i, 2].axis('off')

# Add column titles
axs[0, 0].set_title("Image")
axs[0, 1].set_title("Predicted")
axs[0, 2].set_title("Ground Truth")

plt.tight_layout(rect=[0, 0, 1, 0.97])

# Log the plot to W&B
wandb.log({"10x3 Prediction Grid": wandb.Image(fig)})

# Close the plot to prevent memory issues in notebooks
plt.close(fig)
